<a href="https://colab.research.google.com/github/Sairaj-97/Deep-Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow import keras

In [2]:
(ds_train,ds_test),ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]','train[80%:]'],
    with_info=True,
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tf_flowers/incomplete.UNR4NB_3.0.1/tf_flowers-train.tfrecord*...:   0%|   …

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
print(ds_info)

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_dir='/root/tensorflow_datasets/tf_flowers/3.0.1',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)


In [4]:
for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)

Image shape: (333, 500, 3)
Label: tf.Tensor(2, shape=(), dtype=int64)


In [5]:
# Preprocessing function for resizing and normalizing images
def preprocess(image, label):
    # Resize the input image to 128x128 pixels
    # (This ensures all images have the same shape for CNN input)
    image = tf.image.resize(image, (128, 128))

    # Convert image data type from uint8 (0–255) to float32
    image = tf.cast(image, tf.float32)

    # Normalize pixel values from [0, 255] to [0, 1]
    # (This helps the model train faster and perform better)
    image = image / 255.0

    # Return the processed image and its corresponding label
    return image, label


In [6]:
# Apply the preprocessing function to each (image, label) pair in the training dataset
# 'map' applies the function and 'num_parallel_calls=AUTOTUNE' uses multiple CPU threads to speed it up
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetching overlaps data preprocessing and model execution for better performance
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

# Apply the same preprocessing steps to the test dataset
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Prefetch the test data as well to ensure smooth evaluation
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

for image, label in ds_train.take(1):
    print("Image shape:", image.shape)
    print("Label:", label)


Image shape: (128, 128, 3)
Label: tf.Tensor(2, shape=(), dtype=int64)


#batching is mandatory when using model.fit() in TensorFlow/Keras.



In [7]:
ds_train = ds_train.batch(32)
ds_test = ds_test.batch(32)

#loading existing trained model

In [8]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3),
                                               include_top=False,
                                               weights='imagenet')

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


#freeze the training of this base model

In [9]:
base_model.trainable=False

#making our dense layers and attaching the pretrained model

In [14]:
inputs = keras.Input(shape=(128, 128, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.MaxPool2D()(x)
# Flatten the output before the dense layer
x = keras.layers.Flatten()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(5,activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [15]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [16]:
model.fit(ds_train,epochs=5)

Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 19s 89ms/step - accuracy: 0.6488 - loss: 1.2844
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9072 - loss: 0.2897
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9357 - loss: 0.1947
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9703 - loss: 0.0803
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9787 - loss: 0.0761


In [18]:
loss,accuracy=model.evaluate(ds_test)

23/23 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - accuracy: 0.8667 - loss: 0.5682


In [19]:
print(accuracy)

0.8692098259925842
